In [1]:
import torch, math, time, argparse, os
import random, dataset, utils, losses, net
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from dataset.market import Market
from net.resnet import *
from net.googlenet import *
from net.bn_inception import *

from tqdm import *

### Loading the model and the data

In [2]:
dataset = dataset.load('market', '../ITCS-5145-CV/learning/Market-1501-v15.09.15/', 'eval', 
                           transform = dataset.utils.make_transform(
                                                                    is_train = False, 
                                                                    is_inception = False
                            )
                       )

In [3]:
dl_ev = torch.utils.data.DataLoader(
        dataset,
        batch_size = 32,
        shuffle = False,
        num_workers = 4,
        pin_memory = True
)

In [5]:
model = Resnet50(embedding_size=512, pretrained=True, is_norm=1, bn_freeze =1).cuda()
model.load_state_dict(torch.load('models/resnet50_best.pth')['model_state_dict'])

/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

### Extracting the embeddings

In [6]:
embeddings, class_labels = [], []
model.eval()
iterator = tqdm(enumerate(dl_ev))
for i, dat in iterator:
    imgs, labels = dat
    imgs = imgs.cuda()
    res = model(imgs)
    embeddings.append(res.cpu().detach().numpy())
    class_labels.append(labels.cpu().numpy())

0it [00:00, ?it/s]/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
335it [00:22, 15.19it/s]


In [7]:
class_labels, embeddings = np.concatenate(class_labels), np.concatenate(embeddings)

In [8]:
embeddings.shape

(10706, 512)

##### Dimensionality Reduction

### Clustering

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(embeddings)

In [10]:
def cosine_sim(x, y):
    return 1 - np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [11]:
agg_clustering = AgglomerativeClustering(n_clusters=500, metric='cosine', linkage='complete')
cluster_labels = agg_clustering.fit_predict(X_scaled)

In [12]:
df = pd.DataFrame({'labels': class_labels, 'cluster': cluster_labels})

In [13]:
df.groupby('labels')['cluster'].agg(['count', 'nunique']).to_csv('results/clustering_results_agg_cosine_resnet_modified_fully_trained.csv')